<a href="https://colab.research.google.com/github/futureCodersSE/python-cyber/blob/main/Creating_a_serverless_function_(hosted_on_AWS_lambda).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The following code will be used to practice creating a serverless function
---

The function will run in the cloud, in an Amazon Web Services (AWS) account using the **lambda** service.

A lambda function is a serverless function (ie it runs independently in the cloud, started by a 'trigger').

Before we upload the code to the cloud, it is important to understand what it does and the 'extra' code needed to ensure that is can run in the cloud.

## Exercise 1 - create and run a simple function
---

The function `add_numbers(num1, num2)` is a simple Python function that, if given two numbers (`num1` and `num2`) will add them together and return the sum.

Here is the code.  

Click on the cell to activate it.  NOTHING WILL HAPPEN because the function has not been 'triggered' by some code outside the function.  BUT it is ready to run once you have activated it.

In [ ]:
def add_numbers(num1, num2):
  sum = num1 + num2
  return sum


### Now **trigger** the function with a function call

In [ ]:
result = add_numbers(3, 5)
print(result)

Here (in the Google Colab), both code parts are needed in order for the function to get its data, run and deliver the result.

---


## Exercise 2 - add a lambda handler which will run the function in the serverless system
---
Once the function is uploaded to the lambda service, it will not be possible just to type the code in order to run it.  We will have to send a request to run it in a different way and the function needs to be able to understand the request.

The following code is a **lambda handler function** that will be able to run the `add_numbers` function once it has been requested from outside.

This code is the interface between the function and the outside world.  It should always be called `lambda_handler` when working with AWS services (other cloud services have different names for this)

The `lambda_handler` function will receive an **event** which will be a **JSON object** which, for this exercise will look like this:

```
{
  "body":{
    "data": {
      "num1":3,
      "num2":5
    }
  }
}
```
The `event` will contain a "body" of `data`.  The "data" will contain the two numbers that the function needs (`num1` and `num2`)

In [ ]:
import json

def lambda_handler(event, context):
  # event will be in the format {"body":{"data":{"num1":3, "num2":5}} where 3 and 5 could be any numbers.
  # FOR SECURITY ALWAYS first check that the "body" key is there to prevent crashes
  if "body" in event.keys():
    request = event["body"]
    if type(request) is not dict:
      request = json.loads(request)
    # check that there was some data in the body, get the values and run the function to get the result
    if request is not None and "data" in request.keys():
      # get the data from the data object
      data = request["data"]
      num1 = data["num1"]
      num2 = data["num2"]
      # now the data has been collected, it can run the add_numbers(num1, num2) function setting return_data to the result, and set the statuscode to 200 (success)
      return_data = add_numbers(num1, num2)
      statuscode = 200
    else:
      return_data = "Unable to get data"
      statuscode = 404
    # now it can return the result in a JSON object with some security settings in the headers
    return {
        'statusCode': statuscode,
        'headers':{
            'Content-Type' : 'application/json',
            'Access-Control-Allow-Headers' : 'Content-Type,X-Api-Key',
            'Access-Control-Allow-Methods' : 'POST',
            'Access-Control-Allow-Origin':'*'
        },
        'body': json.dumps(return_data)
    }

Again - **run the code above to activate it**

THEN for testing purposes, run the code below to run the lambda_handler (which should then run the add_numbers function.

In [ ]:
# set up the data first
request_event = {
  "body":{
    "data": {
      "num1":3,
      "num2":5
    }
  }
}

# run the lambda_handle with a context of None (this is needed if you want to run the function directly but we are going to do this with an API call, which will deal with the context itself)
result = lambda_handler(request_event, None)
print("Status code is: ", result["statusCode"])
print("Result is: ", result["body"])

### Quick explanation of the headers
---

```
'headers':{
            'Content-Type' : 'application/json',
            'Access-Control-Allow-Headers' : 'Content-Type,X-Api-Key',
            'Access-Control-Allow-Methods' : 'POST',
            'Access-Control-Allow-Origin':'*',
        },
```

These provide the configuration that allows the data to be returned from the API gateway (or elsewhere if triggered differently).

`'Content-Type' : 'application/json',` this states that the data to be returned will be in JSON format

` 'Access-Control-Allow-Headers' : 'Content-Type,X-Api-Key',`
This controls the types of headers that are allowed in the request.  In this case, the request can state the content type (in this case it would be 'application/json' as well, to specify an API key.  Other headers would cause a connection error and so can filter out some malicious attacks.

`'Access-Control-Allow-Methods' : 'POST',`
This to only allow POST requests (others including GET, PUT are not accepted)

`'Access-Control-Allow-Origin':'*',`
 This will allow requests from any origin (IP address or domain name).  This is not secure but will allow us to test without knowing the origin for now (a whitelist can be added later so that instead of '*' a list of URLs is used to control access)

# YOUR TASK
---

You are going to make a folder of 3 files containing the code that will make the lambda function work.

## Good practice in creating lambda functions
---

1.  The lambda_handler function should check for as many data errors as possible to minimise the risk of crashing.  A crash could leave a database or other service exposed.  The app should always fail gracefully.  Some of this can be done with try: except: clauses which will be covered later.  

2.  The lambda_handler should sit in a file on its own and should call functions from other files.  This reduces visibility of as many functions as possible in case of crashes or unauthorised access to running code.

3.  All other functions should be imported into the lambda_handler ONLY if that function is called within the handler.

## Instructions
---
1.  On your own device, create a folder called `lambda_files`
2.  In the new folder:
*  Create a text file (use Notepad or other text editor, save as any file type) called `functions.py`
*  Create a text file in the same way and call it `lambda_function.py`
*  Create a text file in the same way can call it `__init__.py`

### Copy this code into the new files

**Note**:  these can't be run together in a Colab notebook because they are specifically written to work in different files.

**functions.py** ( a file containing the functions that are being requested)

```
## functions.py

# a set of mathematical functions
def add_numbers(num1, num2):
  sum = num1 + num2
  return sum
```

**lambda_function.py** (a file containing only the *lambda_handler* function)

```
# lambda_function.py

# handle the request, passing the data to other functions

import json
from functions import add_numbers

def lambda_handler(event, context):
  # event will be in the format {"body":{"data":{{"num1":3, "num2":5}}} where 3 and 5 could be any numbers.
  # FOR SECURITY ALWAYS first check that the "body" key is there to prevent crashes.  If body is a JSON string (rather than object) convert to an object
  if "body" in event.keys():
    request = event["body"]
    if type(request) is not dict:
      request = json.loads(request)
    # check that there was some data in the body, get the values and run the function to get the result
    if request is not None and "data" in request.keys():
      # get the data from the data object
      data = request["data"]
      num1 = data["num1"]
      num2 = data["num2"]
      # now the data has been collected, it can run the add_numbers(num1, num2) function setting return_data to the result, and set the statuscode to 200 (success)
      return_data = add_numbers(num1, num2)
      statuscode = 200
    else:
      return_data = "Unable to get data"
      statuscode = 404
    # now it can return the result in a JSON object with some security settings in the headers
    return {
        'statusCode': statuscode,
        'headers':{
            'Content-Type' : 'application/json',
            'Access-Control-Allow-Headers' : 'Content-Type,X-Api-Key',
            'Access-Control-Allow-Methods' : 'POST',
            'Access-Control-Allow-Origin':'*'
        },
        'body': json.dumps(return_data)
    }
  ```

  
 **Initialisation file**:  `__init__.py`.  This helps with the tracking of files as your folders get bigger and you have sub-folders.  THE FILE SHOULD BE EMPTY (although at advanced stages you may learn to add to it)

  

### Get code ready to upload to Lambda
---

SO a folder (called **lambda_files** here) containing all the code for this lambda function should look like this:

  ```
  lambda_files  
  --> __init__.py  
  --> functions.py  
  --> lambda_function.py  
  ```
  

## The next stage is to try this out in AWS Lambda.
---

For this you will need an AWS account.  You will be asked for credit card details but we will NOT be using any services that will incur a charge and you will be given guidance on how to set up a budgeting system to help prevent costs being added by accident.

**PLEASE BE AWARE that AWS has a wide range of services, not all of them are free from cost.  If you experiment outside the services we are asking you to use, you may be charged.**